In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [67]:
#Let´s combine both datasets to work with them and then split them in train-test sets
df_train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df_full=pd.concat([df_train,df_test],ignore_index=True)

df_full.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [68]:
df_full.isnull().sum()
#We cannot see all of them. Let´s see it manually

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       486
LotArea             0
                 ... 
MoSold              0
YrSold              0
SaleType            1
SaleCondition       0
SalePrice        1459
Length: 81, dtype: int64

In [69]:
#null values in the df
"""
We have to reduce to the minimum the number of unknown variables.
The least to deduce is SalePrice which is the variable that we have
to predict in USD$.
"""

print("Null values in the dataframe: \n")
for i in range(len(df_full.isnull().sum())):
    if df_full[df_full.columns[i]].isnull().sum(): #if a column has null elements
        print("{}: {}".format(df_full.columns[i], df_full[df_full.columns[i]].isnull().sum()))
    else:
        continue

Null values in the dataframe: 

MSZoning: 4
LotFrontage: 486
Alley: 2721
Utilities: 2
Exterior1st: 1
Exterior2nd: 1
MasVnrType: 24
MasVnrArea: 23
BsmtQual: 81
BsmtCond: 82
BsmtExposure: 82
BsmtFinType1: 79
BsmtFinSF1: 1
BsmtFinType2: 80
BsmtFinSF2: 1
BsmtUnfSF: 1
TotalBsmtSF: 1
Electrical: 1
BsmtFullBath: 2
BsmtHalfBath: 2
KitchenQual: 1
Functional: 2
FireplaceQu: 1420
GarageType: 157
GarageYrBlt: 159
GarageFinish: 159
GarageCars: 1
GarageArea: 1
GarageQual: 159
GarageCond: 159
PoolQC: 2909
Fence: 2348
MiscFeature: 2814
SaleType: 1
SalePrice: 1459


In [86]:
#Let´s see how the function in next cell fills the data (quotation marks)
df_full.loc[df_full["SaleType"].notnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [85]:
#Let´s see how the function in next cell fills the data (quotation marks)
df_full.loc[df_full["SaleType"].isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
2489,2490,20,RL,85.0,13770,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2007,NaN,Normal,NaN


In [94]:
"""
Observation, the features that have the lowest ammount of lost information
we will substitute it by the most common feature.
"""

"""
Observing the explanation provided by kaggle. There are many "NaN" cells for some columns.
For example:

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
       
These variables are:

Alley: NA 	No alley access    Alley: 2721
BsmtQual: NA	No Basement    BsmtQual: 81
BsmtCond: NA	No Basement    BsmtCond: 82
BsmtExposure: NA	No Basement BsmtExposure: 82
BsmtFinType1: NA	No Basement BsmtFinType1: 79
BsmtFinType2: NA	No Basement BsmtFinType2: 80
FireplaceQu: NA	No Fireplace    FireplaceQu: 1420
GarageType: NA	No Garage       GarageType: 157
GarageFinish: NA	No Garage   GarageFinish: 159
GarageQual: NA	No Garage       GarageQual: 159
GarageCond: NA	No Garage       GarageCond: 159
PoolQC: NA	No Pool             PoolQC: 2909
Fence: NA	No Fence            Fence: 2348
MiscFeature: NA 	None        MiscFeature: 2814

Which they do not mean that there isn´t data. They only mean that they don´t have it.
For these variables the best option would be to introduce an escaler in their column
being NA the worst of them all (0) up to the best quality (3-4) depending on the feature

The lost variables are:

LotFrontage: 486    We will guess it
MasVnrType: 24      We will put the most common one
MasVnrArea: 23      We will put the most common one


GarageYrBlt: 159    As expected, these are the houses without garage (We will put year 0)
SalePrice: 1459     The one to predict
"""

for i in df_full.columns:
    #for every column name
    if (df_full['{}'.format(i)].isnull().sum()!=0)&(df_full['{}'.format(i)].isnull().sum()<=25):
        #if the total nulls is different and less than 5
        df_full['{}'.format(i)].fillna(df_full['{}'.format(i)].mode()[0], inplace=True)
        #the df_full[index] fill the nulls with the most common mode

In [97]:
pd.DataFrame(df_full.loc[2489]).T
#The format is correct (without quotations)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
2489,2490,20,RL,85.0,13770,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2007,WD,Normal,NaN


In [98]:
#We run again the data to check that it was correct
print("Null values in the dataframe: \n")
for i in range(len(df_full.isnull().sum())):
    if df_full[df_full.columns[i]].isnull().sum(): #if a column has null elements
        print("{}: {}".format(df_full.columns[i], df_full[df_full.columns[i]].isnull().sum()))
    else:
        continue

Null values in the dataframe: 

LotFrontage: 486
Alley: 2721
BsmtQual: 81
BsmtCond: 82
BsmtExposure: 82
BsmtFinType1: 79
BsmtFinType2: 80
FireplaceQu: 1420
GarageType: 157
GarageYrBlt: 159
GarageFinish: 159
GarageQual: 159
GarageCond: 159
PoolQC: 2909
Fence: 2348
MiscFeature: 2814
SalePrice: 1459


In [129]:
#Modify columns with levels 0-5:

print("BsmtQual: ", df_full["BsmtQual"].unique())
print("BsmtCond: ", df_full["BsmtCond"].unique())
print("BsmtExposure: ", df_full["BsmtExposure"].unique())
print("BsmtFinType1: ", df_full["BsmtFinType1"].unique())
print("BsmtFinType2: ", df_full["BsmtFinType2"].unique())
print("FireplaceQu: ", df_full["FireplaceQu"].unique())
print("GarageType: ", df_full["GarageType"].unique())
print("GarageYrBlt: ", df_full["GarageYrBlt"].unique())
print("GarageFinish: ",df_full["GarageFinish"].unique())
print("GarageQual: ", df_full["GarageQual"].unique())
print("GarageCond: ", df_full["GarageCond"].unique())
print("PoolQC: ", df_full["PoolQC"].unique())
print("Fence: ", df_full["Fence"].unique())

BsmtQual:  ['Gd' 'TA' 'Ex' nan 'Fa']
BsmtCond:  ['TA' 'Gd' nan 'Fa' 'Po']
BsmtExposure:  ['No' 'Gd' 'Mn' 'Av' nan]
BsmtFinType1:  ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
BsmtFinType2:  ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
FireplaceQu:  [nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']
GarageType:  ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
GarageYrBlt:  [2003. 1976. 2001. 1998. 2000. 1993. 2004. 1973. 1931. 1939. 1965. 2005.
 1962. 2006. 1960. 1991. 1970. 1967. 1958. 1930. 2002. 1968. 2007. 2008.
 1957. 1920. 1966. 1959. 1995. 1954. 1953.   nan 1983. 1977. 1997. 1985.
 1963. 1981. 1964. 1999. 1935. 1990. 1945. 1987. 1989. 1915. 1956. 1948.
 1974. 2009. 1950. 1961. 1921. 1900. 1979. 1951. 1969. 1936. 1975. 1971.
 1923. 1984. 1926. 1955. 1986. 1988. 1916. 1932. 1972. 1918. 1980. 1924.
 1996. 1940. 1949. 1994. 1910. 1978. 1982. 1992. 1925. 1941. 2010. 1927.
 1947. 1937. 1942. 1938. 1952. 1928. 1922. 1934. 1906. 1914. 1946. 1908.
 1929. 1933. 1917. 1896. 1895. 2207. 1943. 1919.]
Ga

In [141]:
# I use this way because I couldn´t find a way to automate each especific cell type
#(in decreasing importance order) per column

#BsmtQual Column

df_full.loc[df_full.BsmtQual=="Ex", "BsmtQual"]=5
df_full.loc[df_full.BsmtQual=="Gd", "BsmtQual"]=4
df_full.loc[df_full.BsmtQual=="TA", "BsmtQual"]=3
df_full.loc[df_full.BsmtQual=="Fa", "BsmtQual"]=2
df_full.loc[df_full.BsmtQual=="Po", "BsmtQual"]=1
df_full.loc[df_full.BsmtQual.isnull()]=0

#BsmtCond Column

df_full.loc[df_full.BsmtCond=="Ex", "BsmtCond"]=5
df_full.loc[df_full.BsmtCond=="Gd", "BsmtCond"]=4
df_full.loc[df_full.BsmtCond=="TA", "BsmtCond"]=3
df_full.loc[df_full.BsmtCond=="Fa", "BsmtCond"]=2
df_full.loc[df_full.BsmtCond=="Po", "BsmtCond"]=1
df_full.loc[df_full.BsmtCond.isnull()]=0

#BsmtExposure Column

df_full.loc[df_full.BsmtExposure=="Gd", "BsmtExposure"]=4
df_full.loc[df_full.BsmtExposure=="Av", "BsmtExposure"]=3
df_full.loc[df_full.BsmtExposure=="Mn", "BsmtExposure"]=2
df_full.loc[df_full.BsmtExposure=="No", "BsmtExposure"]=1
df_full.loc[df_full.BsmtExposure.isnull()]=0

#BsmtFinType1 Column

df_full.loc[df_full.BsmtFinType1=="GLQ", "BsmtFinType1"]=6
df_full.loc[df_full.BsmtFinType1=="ALQ", "BsmtFinType1"]=5
df_full.loc[df_full.BsmtFinType1=="BLQ", "BsmtFinType1"]=4
df_full.loc[df_full.BsmtFinType1=="Rec", "BsmtFinType1"]=3
df_full.loc[df_full.BsmtFinType1=="LwQ", "BsmtFinType1"]=2
df_full.loc[df_full.BsmtFinType1=="Unf", "BsmtFinType1"]=1
df_full.loc[df_full.BsmtFinType1.isnull()]=0

#BsmtFinType2 Column

df_full.loc[df_full.BsmtFinType2=="GLQ", "BsmtFinType2"]=6
df_full.loc[df_full.BsmtFinType2=="ALQ", "BsmtFinType2"]=5
df_full.loc[df_full.BsmtFinType2=="BLQ", "BsmtFinType2"]=4
df_full.loc[df_full.BsmtFinType2=="Rec", "BsmtFinType2"]=3
df_full.loc[df_full.BsmtFinType2=="LwQ", "BsmtFinType2"]=2
df_full.loc[df_full.BsmtFinType2=="Unf", "BsmtFinType2"]=1
df_full.loc[df_full.BsmtFinType2.isnull()]=0

#FireplaceQu Column

df_full.loc[df_full.FireplaceQu=="Ex", "FireplaceQu"]=5
df_full.loc[df_full.FireplaceQu=="Gd", "FireplaceQu"]=4
df_full.loc[df_full.FireplaceQu=="TA", "FireplaceQu"]=3
df_full.loc[df_full.FireplaceQu=="Fa", "FireplaceQu"]=2
df_full.loc[df_full.FireplaceQu=="Po", "FireplaceQu"]=1
df_full.loc[df_full.FireplaceQu.isnull()]=0

#GarageType Column

df_full.loc[df_full.GarageType=="2Types", "GarageType"]=6
df_full.loc[df_full.GarageType=="Attchd", "GarageType"]=5
df_full.loc[df_full.GarageType=="Basment", "GarageType"]=4
df_full.loc[df_full.GarageType=="BuiltIn", "GarageType"]=3
df_full.loc[df_full.GarageType=="CarPort", "GarageType"]=2
df_full.loc[df_full.GarageType=="Detchd", "GarageType"]=1
df_full.loc[df_full.GarageType.isnull()]=0

#GarageYrBlt Column

#df_full.loc[df_full.GarageYrBlt.isnull()]=0

#GarageFinish Column

df_full.loc[df_full.GarageFinish=="Fin", "GarageFinish"]=3
df_full.loc[df_full.GarageFinish=="RFn", "GarageFinish"]=2
df_full.loc[df_full.GarageFinish=="Unf", "GarageFinish"]=1
df_full.loc[df_full.GarageFinish.isnull()]=0

#GarageQual Column

df_full.loc[df_full.GarageQual=="Ex", "GarageQual"]=5
df_full.loc[df_full.GarageQual=="Gd", "GarageQual"]=4
df_full.loc[df_full.GarageQual=="TA", "GarageQual"]=3
df_full.loc[df_full.GarageQual=="Fa", "GarageQual"]=2
df_full.loc[df_full.GarageQual=="Po", "GarageQual"]=1
df_full.loc[df_full.GarageQual.isnull()]=0

#GarageCond Column

df_full.loc[df_full.GarageCond=="Ex", "GarageCond"]=5
df_full.loc[df_full.GarageCond=="Gd", "GarageCond"]=4
df_full.loc[df_full.GarageCond=="TA", "GarageCond"]=3
df_full.loc[df_full.GarageCond=="Fa", "GarageCond"]=2
df_full.loc[df_full.GarageCond=="Po", "GarageCond"]=1
df_full.loc[df_full.GarageCond.isnull()]=0

#PoolQC Column

df_full.loc[df_full.PoolQC=="Ex", "PoolQC"]=4
df_full.loc[df_full.PoolQC=="Gd", "PoolQC"]=3
df_full.loc[df_full.PoolQC=="TA", "PoolQC"]=2
df_full.loc[df_full.PoolQC=="Fa", "PoolQC"]=1
df_full.loc[df_full.PoolQC.isnull()]=0

#Fence Column

df_full.loc[df_full.Fence=="GdPrv", "Fence"]=4
df_full.loc[df_full.Fence=="MnPrv", "Fence"]=3
df_full.loc[df_full.Fence=="GdWo", "Fence"]=2
df_full.loc[df_full.Fence=="MnWw", "Fence"]=1
df_full.loc[df_full.Fence.isnull()]=0